In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import re
import sys
import csv
import time
import heapq
import tweepy # https://github.com/tweepy/tweepy
import numpy as np
import pandas as pd
import configparser
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, LSTM, Dropout, GRU, TimeDistributed, BatchNormalization
from keras.layers import CuDNNLSTM 
from keras.layers.core import Dense, Activation, Dropout, RepeatVector

from keras.utils import np_utils
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint
# from keras.optimizers import RMSprop

In [ ]:
Handler = 'elonmusk'
path = 'drive/MyDrive/Colab Notebooks/COE494_Project'

In [ ]:
def authenticate(path = 'drive/MyDrive/Colab Notebooks/COE494_Project/'):
  # read config
  config = configparser.ConfigParser()
  config.read(path + 'config.ini')

  api_key = str(config['twitter']['api_key'])
  api_key_secret = str(config['twitter']['api_key_secret'])

  access_token = str(config['twitter']['access_token'])
  access_token_secret = str(config['twitter']['access_token_secret'])

  # authenticate
  auth = tweepy.OAuthHandler(api_key, api_key_secret)
  auth.set_access_token(access_token, access_token_secret)

  return tweepy.API(auth, wait_on_rate_limit = True)

In [ ]:
api = authenticate()

In [ ]:
# Tweet text pre-processing
def clean_tweet(tweet):
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [ ]:
def get_all_tweets(handler):
    # Twitter only allows access to a users most recent 3240 tweets with this method
    print(f'Grabbing @{handler}\'s Tweets')
    #initialize a list to hold all the tweepy Tweets
    all_tweets = []  
    
    # make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = handler, count = 200, include_rts = False, tweet_mode = 'extended')
    
    # save most recent tweets
    all_tweets.extend(new_tweets)
    
    # save the id of the oldest tweet less one
    oldest = all_tweets[-1].id - 1
    
    # keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:        
        # all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = handler, count=200, max_id = oldest, include_rts = False, tweet_mode = 'extended')
        # save most recent tweets
        all_tweets.extend(new_tweets)        
        # update the id of the oldest tweet less one
        oldest = all_tweets[-1].id - 1
        
    print(f"{len(all_tweets)} tweets downloaded...")    
    # transform the tweepy tweets into a 2D array that will populate the csv 
    out_tweets = [[tweet.id_str, tweet.created_at, tweet.full_text] for tweet in all_tweets]
    df = pd.DataFrame (out_tweets, columns = ["id", "time", "tweet"])
    df.to_csv(path + '/data/' + handler+'.csv')
    return df

In [ ]:
# tweets = get_all_tweets(Handler)
tweets = pd.read_csv(path + "/data/elonmusk.csv")
cleaned_tweets = pd.DataFrame([clean_tweet(tweet) for tweet in tweets.tweet], columns = ['tweet'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,4,9,14,16,17,19,22,24,25,26,31,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Removing null and empty rows 
cleaned_tweets.tweet.replace('', np.nan, inplace=True)
cleaned_tweets.dropna(inplace = True)
cleaned_tweets

,tweet
0,please ignore prior tweets as that was someone...
1,so true
2,if you ever wanted know real truth about moon ...
3,walked around neighborhood recently rebuilt wi...
4,it was xmas so we brought presents kids at orp...
...,...
34870,reminds me when i hex edited ultima v get out ...
34871,yay switzerland
34872,there is no way be touch with voters when you ...
34874,let s make roaring 20 s happen


In [ ]:
raw_tweets_text = ' '.join(cleaned_tweets["tweet"])

In [ ]:
! pip install aitextgen

In [ ]:
! pip install -q gpt-2-simple

In [ ]:
from aitextgen import aitextgen 
ai = aitextgen(tf_gpt2 ='355M', to_gpu = True )

In [ ]:
# pd.set_option('display.max_colwidth', None)
cleaned_tweets.to_csv('tweets.txt', columns = ['tweet'], header = False, index = False)

In [ ]:
ai.train('tweets.txt',
         line_by_line = True,
         from_cache = False,
         num_steps = 5000, 
         generate_every = 2500,
         save_every = 2500,
         save_gdrive = False,
         learning_rate = 1e-3,
         fp16 = False,
         batch_size = 1)

  0%|          | 0/33624 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
/usr/local/lib/python3.7/dist-packages/pytorch_lightn

  0%|          | 0/5000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2265: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


2,500 steps reached: saving model to /trained_model
2,500 steps reached: generating sample texts.

5,000 steps reached: saving model to /trained_model
5,000 steps reached: generating sample texts.



In [ ]:
seed = 'Roadster'
preds = ai.generate(1, prompt = seed, temperatur = 0.75, top_p = 0.9)
preds

Roadster is wrong that is what really matters is how long civilization is capable making jump mars this could be very short period time measured decades it took 4 5 billion years



In [ ]:
seed = 'Finally'
preds = ai.generate(1, prompt = seed, temperatur = 0.75, top_p = 0.9)
preds

Finally some left turns at traffic lights found by internal standards but no problem



In [ ]:
from nltk.translate import bleu_score as bleu

def corpus_bleu(generated, references):
    """ Compute similarity between two corpora as measured by
    comparing each sentence of `generated` against all sentences in `references` 
    
    args:
        - generated (List[List[str]]): list of sentences (split into tokens)
        - references (List[List[str]]): list of sentences (split into tokens)
        
    returns:
        - bleu (float)
    """    
    return bleu.corpus_bleu([references for _ in range(len(generated))], generated)
    

In [ ]:
print("Model_4(AITextGen) BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data)))